In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle
from pickle import dump

In [2]:

import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras import utils
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Activation
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras import datasets

from keras.callbacks import LearningRateScheduler
from keras.callbacks import History

from keras import losses
from sklearn.utils import shuffle

Using TensorFlow backend.


In [3]:
# Read in csv
df = pd.read_csv("LouisvilleCleanFinal.csv").drop(['Unnamed: 0'],axis=1)
df.head()

,PrimaryColor,BreedCategory,IntakeStatus,PetAgeCategory,Sex,TopBreed,BinaryOutcome
0,WHITE,PURE,NORMAL,YOUNG,MALE,PIT BULL TERRIER,DENY
1,BLACK,MIX,NORMAL,YOUNG,FEMALE,BORDER COLLIE,TAKE
2,TAN,MIX,NORMAL,YOUNG,MALE,GOLDEN RETRIEVER,TAKE
3,WHITE,PURE,NORMAL,BABY,FEMALE,LABRADOR RETRIEVER,TAKE
4,BLACK,PURE,SICK,YOUNG,MALE,PIT BULL TERRIER,DENY


In [5]:
df.head()

,PrimaryColor,BreedCategory,IntakeStatus,PetAgeCategory,Sex,TopBreed,BinaryOutcome,BinaryOutcome2
0,WHITE,PURE,NORMAL,YOUNG,MALE,PIT BULL TERRIER,DENY,0
1,BLACK,MIX,NORMAL,YOUNG,FEMALE,BORDER COLLIE,TAKE,0
2,TAN,MIX,NORMAL,YOUNG,MALE,GOLDEN RETRIEVER,TAKE,0
3,WHITE,PURE,NORMAL,BABY,FEMALE,LABRADOR RETRIEVER,TAKE,0
4,BLACK,PURE,SICK,YOUNG,MALE,PIT BULL TERRIER,DENY,0


In [9]:
X = df.drop(['BinaryOutcome', 'BinaryOutcome2'], axis="columns")
y = df[['BinaryOutcome2']]

# intakestatus, PetAgeCategory, Sex
X.head()
# y.head()

,PrimaryColor,BreedCategory,IntakeStatus,PetAgeCategory,Sex,TopBreed
0,WHITE,PURE,NORMAL,YOUNG,MALE,PIT BULL TERRIER
1,BLACK,MIX,NORMAL,YOUNG,FEMALE,BORDER COLLIE
2,TAN,MIX,NORMAL,YOUNG,MALE,GOLDEN RETRIEVER
3,WHITE,PURE,NORMAL,BABY,FEMALE,LABRADOR RETRIEVER
4,BLACK,PURE,SICK,YOUNG,MALE,PIT BULL TERRIER


In [10]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

In [ ]:
y = ohe.fit_transform(y).toarray()
y


In [11]:
X = ohe.fit_transform(X).toarray()
X.shape

(41535, 52)

In [17]:
# # If you do a Random Forest, BinaryOutcome should be a 0 or 1.  Then only onehot encode X (see example below)
# # X = ohe.fit_transform(X)

# # df["BinaryOutcome2"] = 0
# # df.loc[df["BinaryOutcome"].str.contains("TAKE"), "BinaryOutcome2"] = 1

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# # Import, initialize, fit and predict
# from sklearn.ensemble import RandomForestClassifier
# rf_model = RandomForestClassifier(n_estimators = 100, random_state = 42) 
# rf_model.fit(X_train, y_train)

# # Test
# predict_y_test = rf_model.predict(X_test)

# # Validate - run accuracy score
# from sklearn import metrics
# print("Accuracy score: ", metrics.accuracy_score(y_test, predict_y_test))

# # pickle
# pickle.dump(rf_model, open("modelonehot.pkl","wb"))

### NN model - Initial model


In [ ]:
import tensorflow
tensorflow.keras.__version__
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=52))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [ ]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
 model.fit(
    X_train,
    y_train,
    epochs=30,
    shuffle=True,
    verbose = 2
)

In [ ]:
# model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
# print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

#### V1 - Result Normal Neural Network - Loss: 0.8209062417960498, Accuracy: 0.6994414329528809
    * model = Sequential()
    * model.add(Dense(units=200, activation='relu', input_dim=52))
    * model.add(Dense(units=100, activation='relu'))
    * model.add(Dense(units=50, activation='relu'))
    * model.add(Dense(units=2, activation='softmax'))

#### V2 - Result Normal Neural Network - Loss: 0.8545986787747528, Accuracy: 0.70329350233078
    * model = Sequential()
    * model.add(Dense(units=200, activation='relu', input_dim=52))
    * model.add(Dense(units=200, activation='relu'))
    * model.add(Dense(units=200, activation='relu'))
    * model.add(Dense(units=2, activation='softmax'))

### V3 - Result Normal Neural Network - Loss: 0.820210349807754, Accuracy: 0.7028120160102844
### (Adding more layers, not improving score)
    * model = Sequential()
    * model.add(Dense(units=200, activation='relu', input_dim=52))
    * model.add(Dense(units=200, activation='relu'))
    * model.add(Dense(units=200, activation='relu'))
    * model.add(Dense(units=200, activation='relu'))
    * model.add(Dense(units=200, activation='relu'))
    * model.add(Dense(units=200, activation='relu'))
    * model.add(Dense(units=2, activation='softmax'))

## Hypertuning


In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

### Find best NETWORK WEIGHT INITIALIZATION
    * Result: Best Accuracy for 0.6934 using {'init_mode': 'glorot_uniform'}

In [ ]:
def create_model(init_mode='uniform'):
    # define model
    model = Sequential()
    model.add(Dense(100, kernel_initializer=init_mode, activation='relu', input_dim=52)) 
    model.add(Dense(100, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(2, kernel_initializer=init_mode, activation='relu'))
    # compile model
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

# model.compile(loss='categorical_crossentropy',
#               optimizer=RMSprop(),

In [ ]:
batch_size = 128
epochs = 10

model_CV = KerasClassifier(build_fn=create_model, epochs=epochs,batch_size=batch_size, verbose=1)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 
             'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

In [ ]:
# print results
print(f'Best Accuracy for {grid_result.best_score_:.4} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'mean={mean:.4}, std={stdev:.4} using {param}')

### Find best OPTIMIZER
    * Result - Best Accuracy for 0.6933 using {'optimizer': 'RMSprop'}

In [ ]:
def create_model(optimizer = 'adam', init_mode='uniform'):
    # define model
    model = Sequential()
    model.add(Dense(100, kernel_initializer=init_mode, activation='relu', input_dim=52)) 
    model.add(Dense(100, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(2, kernel_initializer=init_mode, activation='relu'))
    # compile model
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [ ]:
batch_size = 128
epochs = 10

model_CV = KerasClassifier(build_fn=create_model, epochs=epochs,batch_size=batch_size, verbose=1)
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

In [ ]:
# print results
print(f'Best Accuracy for {grid_result.best_score_:.4} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'mean={mean:.4}, std={stdev:.4} using {param}')

### Find best EPOCHS and BATCH SIZE
    * Result: Best Accuracy for 0.6944 using {'batch_size': 60, 'epochs': 10}

In [ ]:
def create_model(optimizer = 'adam', init_mode='uniform'):
    # define model
    model = Sequential()
    model.add(Dense(100, kernel_initializer=init_mode, activation='relu', input_dim=52)) 
    model.add(Dense(100, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(2, kernel_initializer=init_mode, activation='relu'))
    # compile model
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn=create_model, verbose=0)

batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

In [ ]:
# print results
print(f'Best Accuracy for {grid_result.best_score_:.4} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'mean={mean:.4}, std={stdev:.4} using {param}')

### Find best Dropout Regularization
    * Result:

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

def create_model(dropout_rate=0.0, weight_constraint=0):
    # define model
    model = Sequential()
    model.add(Dense(100, input_dim=52, kernel_initializer='uniform', activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='relu'))
    # compile model
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [ ]:
model_CV = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

In [ ]:
# print results
print(f'Best Accuracy for {grid_result.best_score_:.4} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'mean={mean:.4}, std={stdev:.4} using {param}')

### Tune the Number of Neurons in the Hidden Layer

In [ ]:
def create_model(dropout_rate=0.0, weight_constraint=0):
    # define model
    model = Sequential()
    model.add(Dense(100, input_dim=52, kernel_initializer='uniform', activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='relu'))
    # compile model
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

### Save model


* https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594
* https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [ ]:

# dump(rf_model, open('rf_modelTESTMarc.pkl', 'wb'))
# new_input = [[0, 1, 0, 0, 1, 0]]
# with open('rf_modelTESTMarc.pkl', 'rb') as file:  
#     model = pickle.load(file)
# model.predict(new_input) 

In [ ]:
# pd.read_pickle('rf_modelTEST2.pkl')